In [14]:
#Import all dependencies
import datetime as dt
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
import sqlalchemy 
import pymysql
pymysql.install_as_MySQLdb()

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, DateTime,Date
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session
from flask import Flask, jsonify

In [15]:
# The path to our cleaned CSV files
file1 = "clean_hawaii_stations.csv"
file2 = "clean_hawaii_measurements.csv"

In [16]:
#Read first csv file to a dataframe
stations_df = pd.read_csv(file1)
del stations_df["Unnamed: 0"]

In [17]:
#Describe first csv file 
stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [18]:
#Read second csv file to a dataframe
measurements_df = pd.read_csv(file2)
del measurements_df["Unnamed: 0"]

In [19]:
#Describe second csv file 
measurements_df["date"] = pd.to_datetime(measurements_df["date"])
measurements_df["date"].value_counts()
measurements_df.count()

station    18103
date       18103
prcp       18103
tobs       18103
dtype: int64

In [20]:
#construct a base class for declarative class 
Base = declarative_base()

In [21]:
#create Station class that inherits Base class and define its properties (table name & columns)
class Station(Base):
    __tablename__ = "Station"
    station = Column(String(255), primary_key=True)
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [22]:
#create Measurement class that inherits Base class and define its properties (table name & columns)
class Measurement(Base):
    __tablename__ = "Measurement"
    mid = Column(Integer,primary_key=True,autoincrement=True)
    station = Column(String(255))
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)

In [23]:
#Connect to the database if it exists else create the database and connect
engine = create_engine("sqlite:///hawaii.sqlite")
#Access the metadata object in Base Class to create the tables in the database
Base.metadata.create_all(engine)
#Create a session object for this connection
session = Session(bind=engine)

In [11]:
# for row in stations_df:
#     stationinst = Station(
#     station=row["station"],
#     name=row["name"],
#     latitude=row["latitude"],
#     longitude=row["longitude"],
#     elevation=row["elevation"]
#     )
#     session.add(station)
#     session.commit()    
    
#     Check why this gives the errorTypeError: string indices must be integers
#or why write_frames() --does not even need classes

In [25]:
#Write the Stations dataframe 
#The orient='records' is the key of this, it allows to align with the format mentioned in the doc
#to insert in bulks.

listToWrite = stations_df.to_dict(orient='records')
tableToWriteTo = "Station"

metadata = sqlalchemy.schema.MetaData(bind=engine,reflect=True)
table = sqlalchemy.Table(tableToWriteTo, metadata, autoload=True)
# Open the session
Session = sessionmaker(bind=engine)
session = Session()
conn =engine.connect()
# Insert the dataframe into the database in one bulk
conn.execute(table.insert(), listToWrite)

# Commit the changes
session.commit()

# Close the session
session.close()

C:\Users\indir\Anaconda3\envs\PythonData1\lib\site-packages\ipykernel_launcher.py:6: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  


In [27]:
#Inserting Measurement data to database table in bulk
# The orient='records' is the key of this, it allows to align with the format mentioned in the doc to insert in bulks.
listToWrite = measurements_df.to_dict(orient='records')
tableToWriteTo = "Measurement"

metadata = sqlalchemy.schema.MetaData(bind=engine,reflect=True)
table = sqlalchemy.Table(tableToWriteTo, metadata, autoload=True)

# Open the session
Session = sessionmaker(bind=engine)
session = Session()

# Inser the dataframe into the database in one bulk
conn.execute(table.insert(), listToWrite)

# Commit the changes
session.commit()

# Close the session
session.close()


C:\Users\indir\Anaconda3\envs\PythonData1\lib\site-packages\ipykernel_launcher.py:6: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  


In [28]:
#Verify Station table data inserted through querying class
stations = session.query(Station).count()
stations

9

In [29]:
#Verify Measurement table inserted through querying class
measurements = session.query(Measurement).count()
measurements 

18103

In [30]:
#Close the Session
session.close()